In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

## PHYSICAL CONSTANTS 
T      = 298.15
F      = 96485   # C/mol, Faraday's constant
R      = 8.3145  # J/mol-K, ideal gas constant
f      = F/(R*T) # 1/V, normalized Faraday's constant at room temperature

## INDEPENDENT VARIABLES 
### EXPERIMENTALLY CONTROLED
C_red_val  = 1.0    #  mol/L, initial concentration of O. Default = 1.0
C_ox_val   = 0.0    #  mol/L, initial concentration of O. Default = 1.0
S          = 1E-6   #  electrode surface. Default = 1E-6

### SYSTEM DEPENDENT
E_0        = 0.0    #  electrochemical potential. Default = 0.0
D          = 1E-5   #  cm^2/s, O & R diffusion coefficient. Default = 1E-5
n          = 1.0    #  number of electrons transfered. Default = 1
alpha      = 0.5    #  dimensionless charge-transfer coefficient. Default = 0.5
k0         = 1E-2   #  cm/s, electrochemical rate constant. Default = 1E-2
kc         = 1E-3   #  1/s, chemical rate constant. Default = 1E-3

## SIMULATION VARIABLES
# simulation accuracy in time and space
Nt      = 3000  # number of iterations per t_k (pg 790). Default = 500
Nx      = 200    # number of spatial boxes  Default = 200

# key adimensionnated parameters
DM      = 0.45     #DM = D*Dt/Dx**2 numerical adimensionated diffusion parameter. Default = 0.45
Lambda  = 2        #Lambda  = k0*Dx/D numerical adimensionated electron transfer parameter. Fast if 

# physical distances according to the previosu parameters
L_cuve  = math.floor(4.2*Nt**0.5) + 6  
    # longueur physique de la cuve 
    # so that we are always in a diffusion controled system
Dx      = L_cuve/Nx


## Experimental paramteres in the case of a linear sweep voltammetry
E_i = -1.0
E_f = +1.0
v = 0.1


def main_LSV_Non_Nernstian(LLambda):
    Lambda = LLambda
    
    (E, tk) = rampe(E_i, E_f, v)
    print(E(0))
    ## Pas Dx et Dt
    Dt = tk/Nt  

    print("DM = ", DM, "and lambda = ", Lambda)
    
    ## profil de concentration inital
    C_new = np.append([C_ox_val for i in range(Nx)],[C_red_val for i in range(Nx)])

    ## propagation temporelle
    fig, ax = plt.subplots(3)
    (M_new_constant, M_old) = Matrix_constant(Nx)
    I = np.array(())

    for i in range(Nt):
        C_old = C_new
        t = i*Dt 
        M_new = Matrix_E_Non_Nernst_boundaries(M_new_constant, Nx, t, E, Lambda)
        C_new = compute_Cnew(M_new, M_old, C_old)
        I = np.append(I, compute_I(C_new))
        if i % 100 == 0:
            ax[0].plot([j*Dx for j in range(Nx)], C_new[:Nx], label= 'time = %is' %(i*Dt))
            ax[1].plot([j*Dx for j in range(Nx)], C_new[Nx:], label= 'time = %is' %(i*Dt))

    ax[2].plot([E(i*(Dt)) for i in range(Nt)], I)
    ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    ax[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.show()
    
    return(I)
